# Limpieza de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
Df = pd.read_csv("archivos/Casos_Epidemiol_gicos_20250205.csv") #Leemos la base de datos

## Convertir fechas a formato datetime

Df['Fecha_notificación'] = pd.to_datetime(Df["Fecha_notificación"], format='%m/%d/%Y %H:%M:%S %p')
Df['Fecha_contagio'] = pd.to_datetime(Df["Fecha_contagio"], format='%m/%d/%Y %H:%M:%S %p')

# Eliminamos columnas no relevantes para el estudio

Df.pop("Fecha_hospitalización")
Df.pop("Semana_epidemiológica")
Df.pop("Código_Nacionalidad")
Df.pop("Otros_grupos_riesgo")

# Podemos rellenar información nula en ciertos campos no numericos con N/A
Df["Unidad_generadora_datos"] = Df["Unidad_generadora_datos"].fillna("N/A")
Df["Estrato_socioeconómico"] = Df["Estrato_socioeconómico"].fillna("N/A")


#Los años están escritos, como 2,024. vamos a eliminar dichas comas
Df["Año"] = Df["Año"].astype(str).str.replace(",","",regex=False)
Df["Año_contagio"] = Df["Año_contagio"].astype(str).str.replace(",","",regex=False)

#Eliminamos la ultima fila que tiene unos datos nulos

Df.dropna(inplace=True)
print(Df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8635 entries, 0 to 8635
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Fecha_notificación          8635 non-null   datetime64[ns]
 1   Año                         8635 non-null   object        
 2   Edad_paciente               8635 non-null   int64         
 3   Nacionalidad                8635 non-null   object        
 4   Sexo_paciente               8635 non-null   object        
 5   Codigo Departamento         8635 non-null   object        
 6   Código_Municipio            8635 non-null   int64         
 7   Tipo Seguridad Social       8635 non-null   object        
 8   Estrato_socioeconómico      8635 non-null   object        
 9   Discapacidad                8635 non-null   object        
 10  Desplazado                  8635 non-null   object        
 11  Migrante                    8635 non-null   object        
 1

In [ ]:
### Vamos a cambiar el codigo de municipio por el nombre del municipio ###


# 2. Formatear los códigos en el DataFrame original
# ----------------------------
# Aseguramos que ambos códigos sean strings con relleno de ceros:
Df["Codigo Departamento"] = Df["Codigo Departamento"].astype(str).str.zfill(2)
Df["Código_Municipio"] = Df["Código_Municipio"].astype(str).str.zfill(3)

# Creamos una columna temporal que combine ambos códigos para formar la llave completa
Df["full_code"] = Df["Codigo Departamento"].astype(str) + Df["Código_Municipio"].astype(str)

# ----------------------------
# 3. Cargar el CSV de mapeo y crear diccionarios de mapeo
# ----------------------------
# Cargar el CSV de mapeo (ajusta la ruta y codificación si es necesario)
map_df = pd.read_csv("archivos/municipios_tabla_completa.csv", encoding="utf-8-sig")

# Diccionario para departamentos: {código de 2 dígitos: nombre departamento}
dept_map = pd.Series(
    map_df["Nombre Departamento"].values,
    index=map_df["Código Departamento"].astype(str).str.zfill(2)
).to_dict()

# Diccionario para municipios: {código completo (2 dígitos + 3 dígitos): nombre municipio}
mun_map = {}
for _, row in map_df.iterrows():
    dep_code = str(row["Código Departamento"]).zfill(2)
    mun_code = str(row["Código Municipio"]).zfill(3)
    full_key = dep_code + mun_code  # Ejemplo: "05" + "001" = "05001"
    mun_map[full_key] = row["Nombre Municipio"]

# ----------------------------
# 4. Reemplazar los valores en el DataFrame original usando los diccionarios
# ----------------------------
# Reemplazar el código de departamento por el nombre correspondiente
Df["Codigo Departamento"] = Df["Codigo Departamento"].map(dept_map)
# Reemplazar el código de municipio (usando la llave completa) por el nombre del municipio
Df["Código_Municipio"] = Df["full_code"].map(mun_map)

# ----------------------------
# 5. Renombrar las columnas
# ----------------------------
Df.rename(columns={"Codigo Departamento": "Departamento", "Código_Municipio": "Municipio"}, inplace=True)

# Eliminar la columna temporal ya que no es necesaria
Df.drop(columns=["full_code"], inplace=True)

# Mostrar los primeros registros para verificar el resultado
print(Df.head())